In [48]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import os
from itertools import combinations
import torch

In [49]:
model = SentenceTransformer('stsb-roberta-large')

100%|██████████| 1.31G/1.31G [00:48<00:00, 27.1MB/s]


In [7]:
data_path = '/home/manuto/Documents/world_bank/bert_twitter_labor/twitter-labor-data/data/active_learning/evaluation_inference/US/iter_0-convbert-969622-evaluation'

In [10]:
df = pd.read_csv(os.path.join(data_path, 'is_hired_1mo.csv'))
df.head()

,rank,tweet_id,text,score,point,class
0,1,4.670323e+17,Got the job and I start tomorrow,0.974762,0,1
1,2,2.310183e+09,I don't have to work today or tomorrow!!!! Yay!,0.974679,0,0
2,3,3.952050e+17,I start a new school tomorrow.,0.974638,0,0
3,4,1.153144e+18,"i start my first day tomorrow :,)",0.974602,0,1
4,5,2.480771e+17,So i start my new job tomorrow and im excited!,0.974597,0,1


In [12]:
positive_df = df.loc[df['class'] == 1].reset_index(drop=True)
positive_df.head()

,rank,tweet_id,text,score,point,class
0,1,4.670323e+17,Got the job and I start tomorrow,0.974762,0,1
1,4,1.153144e+18,"i start my first day tomorrow :,)",0.974602,0,1
2,5,2.480771e+17,So i start my new job tomorrow and im excited!,0.974597,0,1
3,6,7.906059e+17,I just got hired at another place !,0.974584,0,1
4,7,7.400481e+17,I start my first day at work tomorrow 🤗,0.974469,0,1


In [13]:
positive_tweet_list = positive_df['text'].tolist()

In [30]:
len(list(combinations(positive_tweet_list, 2)))

1770

In [18]:
embeddings = model.encode(positive_tweet_list, convert_to_tensor=True)

In [32]:
len(embeddings)

60

In [19]:
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)

In [39]:
cosine_scores

tensor([[1.0000, 0.5570, 0.8030,  ..., 0.2457, 0.4291, 0.3142],
        [0.5570, 1.0000, 0.5373,  ..., 0.1175, 0.3942, 0.1655],
        [0.8030, 0.5373, 1.0000,  ..., 0.3850, 0.4708, 0.4188],
        ...,
        [0.2457, 0.1175, 0.3850,  ..., 1.0000, 0.3000, 0.2572],
        [0.4291, 0.3942, 0.4708,  ..., 0.3000, 1.0000, 0.2691],
        [0.3142, 0.1655, 0.4188,  ..., 0.2572, 0.2691, 1.0000]])

In [53]:
value = (-torch.sum(cosine_scores)/(len(positive_tweet_list)**2)).item()
value

-0.4568512439727783

In [47]:
#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t {} \t Score: {:.4f}".format(positive_tweet_list[i], positive_tweet_list[j], pair['score']))
    print('\n')

Start my new job today 💸💰 	 Start my new job today 😁🙌 	 Score: 0.9470


I start my first day at work tomorrow 🤗 	 I start my first day at work tomorrow :) 	 Score: 0.9459


So i start my new job tomorrow and im excited! 	 I start my new job tomorrow and I'm actually really excited 	 Score: 0.9336


I got the job and I start Monday :) 	 Got the job! I start Monday! 	 Score: 0.9203


So i start my new job tomorrow and im excited! 	 I start my new job today I’m so excited!!! 	 Score: 0.9093


I start my new job tomorrow and I'm actually really excited 	 I start my new job today I’m so excited!!! 	 Score: 0.9059


Just got a new job!!!😁 	 I just got a new job 😍😍😍 	 Score: 0.9012


I start my new job tomorrow and I'm actually really excited 	 I start my new job tomorrow and I’m very excited and very scared ☺️😅 	 Score: 0.8960


I start my new job tomorrow and I'm actually really excited 	 I start my first job tomorrow I'm excited 	 Score: 0.8922


So i start my new job tomorrow and im excit